# Welcome to the Jupiter notebook 
### Here I will work on the sakila database.
#### The database is already loaded and is located in the data folder of this project.
#### I loaded the database using dlt, and all the steps are described in the README file.

In [1]:
import duckdb
import pandas as pd 
from pathlib import Path 

# giving the path of the database
duckdb_path = "../data/sakila.duckdb"

# connection to the database
conn = duckdb.connect(duckdb_path)

# setting the schema 'staging' for avoid to write staging.tabel every time
conn.execute("SET search_path='staging';")

# test reading the first 3 films
df_films = conn.sql("SELECT * FROM film LIMIT 5;").df()
description = conn.sql("DESC;").df()

df_films    

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,_dlt_load_id,_dlt_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00,1764369431.1997082,L5oieRwKAj7kew
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1764369431.1997082,6RgtLufzkJU6sQ
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1764369431.1997082,xnr7Y29mIxPhnQ
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00,1764369431.1997082,Jo+2n4NdGLglZw
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00,1764369431.1997082,qNA831UpPJMcmA


In [2]:
df_actor = conn.sql("SELECT first_name FROM actor;").df()
df_actor.head()



,first_name
0,PENELOPE
1,NICK
2,ED
3,JENNIFER
4,JOHNNY


In [3]:
description.head()

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False


### Answer to the question a task 1 
### which movie are longer then 3 hors 

In [4]:
long_film = conn.sql("""--sql
    SELECT 
        title,
        length
    FROM
        film
    WHERE 
        length > 180;                
                     """).df()
long_film.count()
long_film.count()

title     39
length    39
dtype: int64

### so we have 39 movies that are longer then 3 hours in sakila database
### now move on with the question b task 1 
### Which movie have the worl "love" in it's title?

In [5]:
love_word_film = conn.sql("""
    SELECT 
        title,
        rating,
        length,
        description
    FROM
        film
    WHERE 
        LOWER (title) LIKE '%love%';                
                     """).df()
love_word_film

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


### Now the question c in task 1 
### calculate decriptive statistics on the lenght column on movie lenght


In [6]:
statistic = conn.sql("""
    SELECT
        MIN(length) AS min_length,
        MAX(length) AS max_length,
        AVG(length) AS avg_length,
        MEDIAN(length) AS median_length
    FROM film
""").df()

statistic


,min_length,max_length,avg_length,median_length
0,46,185,115.272,114.0


### question d in the task 1 
### The 10 most expensive movies to rent per day

In [7]:
expensive_movie = conn.sql("""
    SELECT 
        title,
        ROUND(rental_rate / rental_duration, 3) AS cost_per_day                
    FROM film
    ORDER BY cost_per_day DESC
    LIMIT 10
                           """).df()
expensive_movie

,title,cost_per_day
0,ACE GOLDFINGER,1.663
1,AMERICAN CIRCUS,1.663
2,AUTUMN CROW,1.663
3,BACKLASH UNDEFEATED,1.663
4,BEAST HUNCHBACK,1.663
5,BEHAVIOR RUNAWAY,1.663
6,BILKO ANONYMOUS,1.663
7,CARIBBEAN LIBERTY,1.663
8,CASPER DRAGONFLY,1.663
9,CASUALTIES ENCINO,1.663


### Question e in the task 1
### Top 10 actors have playde 

In [8]:
top_actors = conn.sql("""
    SELECT 
        a.first_name,
        a.last_name,
        COUNT(*) AS film_count
    FROM actor a
    JOIN film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY a.first_name, a.last_name
    ORDER BY film_count DESC
    LIMIT 10
""").df()

top_actors


,first_name,last_name,film_count
0,SUSAN,DAVIS,54
1,GINA,DEGENERES,42
2,WALTER,TORN,41
3,MARY,KEITEL,40
4,MATTHEW,CARREY,39
5,SANDRA,KILMER,37
6,SCARLETT,DAMON,36
7,UMA,WOOD,35
8,ANGELA,WITHERSPOON,35
9,GROUCHO,DUNST,35


### Now let's explore abi t this database

### 1) For exemple which one are the top 10 movie in rental 
 

In [9]:
most_rented = conn.sql("""
    SELECT 
        f.title,
        COUNT(*) AS total_rentals
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    GROUP BY f.title
    ORDER BY total_rentals DESC
    LIMIT 10
""").df()

most_rented


,title,total_rentals
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,RIDGEMONT SUBMARINE,32
3,SCALAWAG DUCK,32
4,FORWARD TEMPLE,32
5,GRIT CLOCKWORK,32
6,JUGGLER HARDLY,32
7,TIMBERLAND SKY,31
8,ROBBERS JOON,31
9,ZORRO ARK,31


### 2) Which store works more?

In [10]:
store_revenue = conn.sql("""
    SELECT 
        s.store_id,
        SUM(p.amount) AS total_revenue
    FROM payment p
    JOIN rental r ON p.rental_id = r.rental_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN store s ON i.store_id = s.store_id
    GROUP BY s.store_id
    ORDER BY total_revenue DESC
""").df()

store_revenue


,store_id,total_revenue
0,2,33726.77
1,1,33679.79


### 3) How many movies there are for each rating?


In [11]:
df_ratings = conn.sql("""
    SELECT 
        rating
    FROM
        film
                      ;""").df()
df_ratings["rating"].value_counts()

rating
PG-13    223
NC-17    210
R        195
PG       194
G        178
Name: count, dtype: int64

### 4) How much the store 2 have gained in may?

In [12]:
# select all the payment in the second store in dicember
df_may_range_store_2 = conn.sql("""
    SELECT 
        p.amount,
        p.payment_date
    FROM payment p
    JOIN rental r ON p.rental_id = r.rental_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    WHERE i.store_id = 2
      AND p.payment_date >= DATE '2005-05-02'
      AND p.payment_date <  DATE '2005-05-31'
""").df()

df_may_range_store_2.head()


,amount,payment_date
0,2.99,2005-05-25 11:30:37
1,0.99,2005-05-28 10:35:23
2,2.99,2005-05-29 22:43:55
3,0.99,2005-05-29 07:25:16
4,4.99,2005-05-25 08:43:32


### Now we can calcolate the total profit


In [13]:
tot_may = df_may_range_store_2["amount"].sum()
f"The store 2 in the may month have gaied:  {tot_may: .2f} dollars"

'The store 2 in the may month have gaied:   2017.00 dollars'

#### There is another thing we can do to optimize our workflow and use pandas commands more efficiently, without losing the ability to run SQL queries through conn.sql().
#### We can create an empty dictionary, and since we have already defined description as DESC;, this gives us all the table names, which we can use as keys in the new dictionary.

#### Then, using a for loop, we read each table and run a query like SELECT * FROM {name}, storing all the results inside the dictionary.

#### Essentially, we are copying the entire sakila database—with all its data—into a simple Python dictionary. This allows us to perform analysis much faster using pandas/Python syntax.

In [14]:
dfs = {}
for name in description["name"]:
    dfs[name] = conn.sql(f"SELECT * FROM {name}").df()
dfs.keys()

dict_keys(['_dlt_loads', '_dlt_pipeline_state', '_dlt_version', 'actor', 'address', 'category', 'city', 'country', 'customer', 'film', 'film_actor', 'film_category', 'inventory', 'language', 'payment', 'rental', 'staff', 'store'])

#### Now operation like dataframe inspection will be much more easy for example:

In [15]:
dfs['film'].head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,_dlt_load_id,_dlt_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00,1764369431.1997082,L5oieRwKAj7kew
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1764369431.1997082,6RgtLufzkJU6sQ
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1764369431.1997082,xnr7Y29mIxPhnQ
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00,1764369431.1997082,Jo+2n4NdGLglZw
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00,1764369431.1997082,qNA831UpPJMcmA


In [16]:
dfs['actor'].sample(3)


,actor_id,first_name,last_name,last_update,_dlt_load_id,_dlt_id
53,54.0,PENELOPE,PINKETT,2021-03-06 15:51:59,1764369431.1997082,0VY1sBD+kQojdQ
82,83.0,BEN,WILLIS,2021-03-06 15:51:59,1764369431.1997082,GEeO1xZ8yRy7MQ
163,164.0,HUMPHREY,WILLIS,2021-03-06 15:52:00,1764369431.1997082,AdoguTBEblfZnQ


In [17]:
dfs['inventory'].head(3)

,inventory_id,film_id,store_id,last_update,_dlt_load_id,_dlt_id
0,1,1,1,2021-03-06 15:52:08,1764369431.1997082,faOdzkj3UX6iHw
1,2,1,1,2021-03-06 15:52:08,1764369431.1997082,yYXk4i6vk3f19A
2,3,1,1,2021-03-06 15:52:08,1764369431.1997082,MJfp9GgXi1b23g


#   Cool!!!
## Let's go to the task 2 now 
###